In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 18:02:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet

In [4]:
df = spark.read.parquet("fhv_tripdata_2021-01.parquet")

In [5]:
#df.write.csv("fhv_tripdata_2021-01.csv", header=True)

In [6]:
import pandas as pd

In [7]:
# df = pd.read_parquet("fhv_tripdata_2021-01.parquet")
# df.to_csv("fhv_tripdata_2021-01.csv", index=False)

In [8]:
!wc -l fhv_tripdata_2021-01.csv

1154113 fhv_tripdata_2021-01.csv


In [9]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2021-01.csv')

In [10]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [11]:
!head -n 1001 fhv_tripdata_2021-01.csv > head.csv

In [12]:
!wc -l head.csv
!head -n 10 head.csv

1001 head.csv
dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,,,,B00009
B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,,,,B00009
B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,,,,B00013
B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,,72.0,,B00037
B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,,61.0,,B00037
B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,,71.0,,B00037
B00037,2021-01-01 00:18:12,2021-01-01 00:30:04,,91.0,,B00037
B00037,2021-01-01 00:36:15,2021-01-01 00:45:08,,39.0,,B00037
B00037,2021-01-01 00:55:04,2021-01-01 01:13:02,,37.0,,B00037


In [13]:
df_pandas = pd.read_csv('head.csv')

In [14]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [16]:
from pyspark.sql import types

In [17]:
schema = types.StructType([
    types.StructField('dispatching_base_num',types.StringType(), True), 
    types.StructField('pickup_datetime',types.TimestampType(), True), 
    types.StructField('dropOff_datetime',types.TimestampType(), True), 
    types.StructField('PUlocationID',types.IntegerType(), True), 
    types.StructField('DOlocationID',types.IntegerType(), True), 
    types.StructField('SR_Flag',types.DoubleType(), True), 
    types.StructField('Affiliated_base_number',types.StringType(), True)
])

In [20]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2021-01.csv')

In [25]:
df.dtypes

[('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropOff_datetime', 'timestamp'),
 ('PUlocationID', 'int'),
 ('DOlocationID', 'int'),
 ('SR_Flag', 'double'),
 ('Affiliated_base_number', 'string')]

In [26]:
df.repartition(24)

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropOff_datetime: timestamp, PUlocationID: int, DOlocationID: int, SR_Flag: double, Affiliated_base_number: string]

In [29]:
df.write.parquet('fhv/2021/01/', mode='overwrite')

In [31]:
df = spark.read.parquet('fhv/2021/01/')

In [33]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [34]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2021-01-01 00:27:00|2021-01-01 00:44:00|        null|        null|   null|                B00009|
|              B00009|2021-01-01 00:50:00|2021-01-01 01:07:00|        null|        null|   null|                B00009|
|              B00013|2021-01-01 00:01:00|2021-01-01 01:51:00|        null|        null|   null|                B00013|
|              B00037|2021-01-01 00:13:09|2021-01-01 00:21:26|        null|        null|   null|                B00037|
|              B00037|2021-01-01 00:38:31|2021-01-01 00:53:44|        null|        null|   null|                B00037|
|              B00037|2021-01-01 00:59:0

In [37]:
df.select('dispatching_base_num','pickup_datetime', 'dropOff_datetime','PUlocationID','DOlocationID') \
    .filter(df.dispatching_base_num == 'B00009') \
    .show()

+--------------------+-------------------+-------------------+------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|
+--------------------+-------------------+-------------------+------------+------------+
|              B00009|2021-01-01 00:27:00|2021-01-01 00:44:00|        null|        null|
|              B00009|2021-01-01 00:50:00|2021-01-01 01:07:00|        null|        null|
|              B00009|2021-01-01 01:57:00|2021-01-01 02:41:00|        null|        null|
|              B00009|2021-01-01 01:45:00|2021-01-01 01:56:00|        null|        null|
|              B00009|2021-01-01 01:06:00|2021-01-01 01:18:00|        null|        null|
|              B00009|2021-01-01 03:36:00|2021-01-01 03:56:00|        null|        null|
|              B00009|2021-01-01 05:32:00|2021-01-01 05:52:00|        null|        null|
|              B00009|2021-01-01 06:40:00|2021-01-01 06:52:00|        null|        null|
|              B00009

In [39]:
from pyspark.sql import functions as F

In [43]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropOff_date', F.to_date(df.dropOff_datetime)) \
    .select('pickup_date', 'dropOff_date','PUlocationID','DOlocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropOff_date|PUlocationID|DOlocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|        null|
| 2021-01-01|  2021-01-01|        null|       

In [58]:
def crazy_function(dispatching_base_num):
    num = int(dispatching_base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [59]:
crazy_function('B00009')

'a/009'

In [60]:
crazy_function_udf = F.udf(crazy_function, returnType=types.StringType())

In [61]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropOff_date', F.to_date(df.dropOff_datetime)) \
    .withColumn('base_id',crazy_function_udf(df.dispatching_base_num)) \
    .select('base_id','pickup_date', 'dropOff_date','PUlocationID','DOlocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropOff_date|PUlocationID|DOlocationID|
+-------+-----------+------------+------------+------------+
|  a/009| 2021-01-01|  2021-01-01|        null|        null|
|  a/009| 2021-01-01|  2021-01-01|        null|        null|
|  e/00d| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/025| 2021-01-01|  2021-01-01|        null|        null|
|  e/05f| 2021-01-01|  2